# Import

In [2]:
%matplotlib inline
import pandas as pd

from requests_html import HTMLSession
from bs4 import BeautifulSoup
import json

# Thu thập dữ liệu
- Nguồn dữ liệu: https://www.cars-data.com/en/

In [ ]:
url = 'https://www.cars-data.com/en/all-cars.html'
Session = HTMLSession()
res = Session.get(url)
s = HTMLSession()

In [ ]:
link_car_urls=[] # List chứa các link xe chưa tính các mẫu của mỗi xe
for html in res.html: #Get link của các xe ở tất cả trang
    cars_one_page = html.find('section.models > div.col-4')
    #print(html.url)
    for car in cars_one_page: #Get link của các xe ở một trang
        url = car.find('a', first = True).attrs['href']
        link_car_urls.append(url)
        
df = pd.DataFrame({'urls': link_car_urls}) # Ghi vào file link_car_urls.csv
df.to_csv("link_car_urls.csv", sep = '\t')

In [ ]:
len(link_car_urls)

In [ ]:
all_link_car_urls=[] #List chứa tất cả các xe bao gồm nhiều mẫu của từng xe
for car_url in link_car_urls:
    r = s.get(car_url)
    section_types = r.html.find('section.types', first = True)
    urls = section_types.find('h2 > a')
    for url in urls: #Duyệt vào trong từng link xe ở list car_urls
        if (url.attrs['href'] != ""):
            all_link_car_urls.append(url.attrs['href'])
    print(len(all_link_car_urls), end='\r')

In [ ]:
len(all_link_car_urls)

In [ ]:
df = pd.DataFrame({'all_models_cars': all_link_car_urls})
df.to_csv("all_models_cars.csv", sep = '\t')

In [ ]:
pd.set_option('display.max_colwidth', None)
all_cars=pd.read_csv('all_models_cars.csv', sep = '\t')['all_models_cars']
all_cars

In [ ]:
# Hàm này là 80% code tham khảo.
def get_detail_car_info(list_url, file_out_name):
    
    file_out = open(file_out_name, 'w', encoding='utf-8')
    file_out.write(f'url\tname\tmodel\tbrand\teLabel\tbodyType\tlength\theight\twidth\tweight\tweightTotal\temissionsCO2\tmodelDate\tfuelType\tnumberOfAxles\tnumberOfDoors\tnumberOfForwardGears\tseatingCapacity\tvehicleTransmission\tcargoVolume\troofLoad\taccelerationTime\tdriveWheelConfiguration\tfuelCapacity\tfuelConsumption\tspeed\tpayload\ttrailerWeight\tvEengineType\tvEfuelType\tvEengineDisplacement\tvEenginePower\ttorque\tprice\n')
    count=0
    for url in list_url:
        r = s.get(url)
        #print(r)
        # Get
        dts = r.html.find('dt.col-6')
        dds = r.html.find('dd.col-6')
        price = 0.0
        eLabel = 'NULL'
        length = 0.0
        for i in range(len(dts)):
            if (dts[i].text == 'price:'):
                price = dds[i].text.split()[-1]
            if (dts[i].text == 'energy label:'):
                eLabel = dds[i].text
            if (dts[i].text == 'length:'):
                length = dds[i].text.split()[0]

        soup = BeautifulSoup(r.html.html, 'html.parser')
        js = json.loads(soup.select_one("script[type='application/ld+json']").string)

        name = js.get('name', "NA")
        bodyType = js.get('bodyType', "NA")
        emissionsCO2 = js.get('emissionsCO2', "NA")
        modelDate = js.get('modelDate', "NA")
        fuelType = js.get('fuelType', "NA")
        numberOfAxles = js.get('numberOfAxles', "NA")
        numberOfDoors = js.get('numberOfDoors', "NA")
        numberOfForwardGears = js.get('numberOfForwardGears', "NA")
        seatingCapacity = js.get('seatingCapacity', "NA")
        vehicleTransmission = js.get('vehicleTransmission', "NA")
        model = js.get('model', "NA")
        url  = js.get('url', "NA")
        cargoVolume = js.get('cargoVolume', "NA")
        roofLoad = js.get('roofLoad', "NA")
        accelerationTime = js.get('accelerationTime', "NA")
        driveWheelConfiguration  = js.get('driveWheelConfiguration', {}).get('name', "NA")
        fuelCapacity = js.get('fuelCapacity', "NA")
        fuelConsumption = js.get('fuelConsumption', "NA")
        speed = js.get('speed', "NA")
        payload = js.get('payload', "NA")
        trailerWeight = js.get('trailerWeight', "NA")
        vEengineType = js.get('vehicleEngine', {}).get('engineType', "NA")
        vEfuelType = js.get('vehicleEngine', {}).get('fuelType', "NA")
        vEengineDisplacement = js.get('vehicleEngine', {}).get('engineDisplacement', "NA")
        vEenginePower = js.get('vehicleEngine', {}).get('enginePower', "NA")
        torque = js.get('vehicleEngine', {}).get('torque', "NA")
        weightTotal = js.get('weightTotal', "NA")
        wheelbase = js.get('wheelbase', "NA")
        height = js.get('height', "NA")
        brand = js.get('manufacturer', {}).get('name', "NA")
        weight = js.get('weight', "NA")
        width = js.get('width', "NA")
        file_out.write(f'{url}\t{name}\t{model}\t{brand}\t{eLabel}\t{bodyType}\t{length}\t{height}\t{width}\t{weight}\t{weightTotal}\t{emissionsCO2}\t{modelDate}\t{fuelType}\t{numberOfAxles}\t{numberOfDoors}\t{numberOfForwardGears}\t{seatingCapacity}\t{vehicleTransmission}\t{cargoVolume}\t{roofLoad}\t{accelerationTime}\t{driveWheelConfiguration}\t{fuelCapacity}\t{fuelConsumption}\t{speed}\t{payload}\t{trailerWeight}\t{vEengineType}\t{vEfuelType}\t{vEengineDisplacement}\t{vEenginePower}\t{torque}\t{price}\n')
        count+=1
        print(count,end='\r')
    file_out.close()

In [ ]:
get_detail_car_info(all_cars, 'all_cars.data')